In [2]:
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

In [6]:
batches_lr_update = 5
epochs = 2
samples = 4000
bs = 128
n_iterations = epochs*samples//bs
n_iterations, n_iterations//batches_lr_update+1

(62, 13)

In [9]:
learning_rates = np.geomspace(1e-4, 1, n_iterations//batches_lr_update+1)

In [10]:
for iteration in range(n_iterations):
    print(learning_rates[iteration//batches_lr_update])

0.0001
0.0001
0.0001
0.0001
0.0001
0.00021544346900318845
0.00021544346900318845
0.00021544346900318845
0.00021544346900318845
0.00021544346900318845
0.00046415888336127773
0.00046415888336127773
0.00046415888336127773
0.00046415888336127773
0.00046415888336127773
0.001
0.001
0.001
0.001
0.001
0.002154434690031882
0.002154434690031882
0.002154434690031882
0.002154434690031882
0.002154434690031882
0.004641588833612777
0.004641588833612777
0.004641588833612777
0.004641588833612777
0.004641588833612777
0.01
0.01
0.01
0.01
0.01
0.021544346900318822
0.021544346900318822
0.021544346900318822
0.021544346900318822
0.021544346900318822
0.046415888336127774
0.046415888336127774
0.046415888336127774
0.046415888336127774
0.046415888336127774
0.1
0.1
0.1
0.1
0.1
0.21544346900318823
0.21544346900318823
0.21544346900318823
0.21544346900318823
0.21544346900318823
0.46415888336127775
0.46415888336127775
0.46415888336127775
0.46415888336127775
0.46415888336127775
1.0
1.0


In [11]:
learning_rates

array([1.00000000e-04, 2.15443469e-04, 4.64158883e-04, 1.00000000e-03,
       2.15443469e-03, 4.64158883e-03, 1.00000000e-02, 2.15443469e-02,
       4.64158883e-02, 1.00000000e-01, 2.15443469e-01, 4.64158883e-01,
       1.00000000e+00])

In [26]:
num_samples = 4000
steps_per_epoch = np.ceil(num_samples / 128)
sample_size=2048
epochs = int(np.ceil(sample_size / float(steps_per_epoch)))
epochs

64

In [23]:
epochs, sample_size, steps_per_epoch, num_samples

(256, 2048, 8.0, 4000)

In [ ]:
np.ceil((len(trainX) / float(config.BATCH_SIZE)))

In [3]:
import argparse
import sys
import os
import torch
import torch.nn as nn
from datetime import date

from age_prediction.models.\
    efficientnet_pytorch_3d import EfficientNet3D as EfNetB0
from age_prediction.data_module import MyDataModule
from age_prediction.module_trainer import ModuleTrainer
from age_prediction.callbacks import ModelCheckpoint, CyclicLR, \
                                      TensorBoardCB, CSVLogger
from age_prediction.metrics import MSE, MAE

In [12]:
import pandas as pd
args = {}
args['database'] = '../datasets'
args['csv_data'] = '../csv_data'
args['side'] = 'L'
args['age_range'] = '[0,70]'
args['gpu'] = 'True'
args['dataParallel'] = 'True'
args['data_aug'] = 'True'
args['snapshot'] = None
args['batch_size'] = '512'
args['loss'] = 'MSE'
args['optimizer'] = 'RMS'
args['num_epochs'] = 2
args['cyclicalLR'] = True
args['weight_decay'] = '1e-5'
args['clr'] = '[-3.1, -1.25]'
args['dropout_rate'] = 0.2
args = pd.DataFrame.from_dict(args, orient='index')
args = args[0]
args

database          ../datasets
csv_data          ../csv_data
side                        L
age_range              [0,70]
gpu                      True
dataParallel             True
data_aug                 True
snapshot                 None
batch_size                512
loss                      MSE
optimizer                 RMS
num_epochs                  2
cyclicalLR               True
weight_decay             1e-5
clr             [-3.1, -1.25]
dropout_rate              0.2
Name: 0, dtype: object

In [13]:
print(args)

# Load images
if args.side == 'L':
    side = "_L"
else:
    side = '_R'

if args.loss == 'MSE':
    loss = nn.MSELoss()
    metrics = [MAE()]
else:
    loss = nn.L1Loss(reduction='mean')
    metrics = [MSE()]

if args.age_range is not None:
    age_range = [int(args.age_range.split(",")[0].split("[")[-1]),
                int(args.age_range.split(",")[-1].split("]")[0])]
    if (age_range[0] == 0) & (age_range[1] == 70):
        train_file = 'train_0-70.csv'
        val_file = 'val_0-70.csv'
    elif (age_range[0] == 70) & (age_range[1] == 100):
        train_file = 'train_70-100.csv'
        val_file = 'val_exp.csv'
    else:
        train_file = 'train_all.csv'
        val_file = 'val_exp.csv'
        print("Using a unknown age range")


dataloader = MyDataModule(database=args.database,
                        csv_data=args.csv_data,
                        side=side,
                        batch=args.batch_size,
                        data_aug=eval(args.data_aug),
                        age_range=age_range,
                        train_file=train_file,
                        val_file=val_file
                        )

dataloader.prepare_data('fit')
dataloader.setup('fit')

train_size = len(dataloader.train.inputs[0])

print("Training with", train_size,
    "evaluating with", len(dataloader.val.inputs[0]))

# Load effNet3D B0
model = EfNetB0.from_name("efficientnet-b0",
                        override_params={
                            'num_classes': 1,
                            'dropout_rate': args.dropout_rate
                        },
                        in_channels=1,
                        )
if eval(args.gpu):
    print("Using GPU")
    device = torch.device('cuda')
    if eval(args.dataParallel):
        if torch.cuda.device_count() > 1:
            print(torch.cuda.device_count(), "GPUs!")
            model = nn.DataParallel(model)
else:
    print("Using cpu")
    device = torch.device('cpu')

model = model.to(device)

if args.optimizer == 'RMS':
    optimizer = torch.optim.RMSprop(model.parameters(),
                                    lr=.256, alpha=0.9,
                                    eps=1e-08, momentum=0.9,
                                    weight_decay=float(args.weight_decay))
elif args.optimizer == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(),
                                lr=.01,
                                weight_decay=float(args.weight_decay))
else:
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=0.01, momentum=0.9,
                                weight_decay=float(args.weight_decay))

if args.snapshot is not None:
    print('Loading model from {}'.format(args.snapshot))
    checkpoint = torch.load(args.snapshot)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    _loss = checkpoint['loss']
    _val_loss = checkpoint['val_loss']
    print("Snapshot trained for {} epochs. \
        Loss: {} and Val loss {}".format(epoch, _loss, _val_loss))

output_folder = 'outputs'
today = date.today()
# dd/mm/YY
d1 = today.strftime("%d-%m-%Y")
output_prefix = d1 + "_age_" + \
    "-".join(str(age_range).split(" ")).replace(',', '') + \
    "_" + args.optimizer + "_wd_" + str(args.weight_decay) + \
    side + "_dp" + str(args.dropout_rate)

print(output_prefix)
callbacks = [CSVLogger(file=os.path.join(output_folder,
                    'logger_' + output_prefix + '.csv'))]

if args.cyclicalLR:
    clr = [float(args.clr.split(",")[0].split("[")[-1]),
        float(args.clr.split(",")[-1].split("]")[0])]
    print('clr limits', clr)
    # 10-3.1, 10-1.25
    num_samples = len(dataloader.train_dataloader().inputs[0])
    step_size = 6*(num_samples//args.batch_size)
    callbacks.append(CyclicLR(base_lr=10**clr[0],
                            max_lr=10**clr[1],
                            mode='triagular2',
                            step_size=step_size
                            )
                    )

callbacks.append(ModelCheckpoint(directory=output_folder,
                                filename='ckpt_' +
                                        output_prefix +
                                        '.pth.tar'))

callbacks.append(TensorBoardCB(log_dir='_'.join(
                                        output_prefix.split("_")[1:]
                                        ), max_img_grid=16))




database          ../datasets
csv_data          ../csv_data
side                        L
age_range              [0,70]
gpu                      True
dataParallel             True
data_aug                 True
snapshot                 None
batch_size                512
loss                      MSE
optimizer                 RMS
num_epochs                  2
cyclicalLR               True
weight_decay             1e-5
clr             [-3.1, -1.25]
dropout_rate              0.2
Name: 0, dtype: object
Preparing data
Setup data
Training with 3500 evaluating with 74
Dropout 0.2
Using GPU
2 GPUs!


RuntimeError: CUDA error: out of memory

In [ ]:
#  Train
trainer = ModuleTrainer(model.to(device))

trainer.compile(loss=loss,
                optimizer=optimizer,
                metrics=metrics,
                callbacks=callbacks)

trainer.fit_loader(dataloader.train_dataloader(),
                dataloader.val_dataloader(),
                num_epoch=int(args.num_epochs),
                cuda_device=True)